# INTRODUCCIÓN

Este set de datos nos propone determinar si un hongo es o no tóxico a partir de sus principales características. Para ello, nos serviremos en paralelo los algoritmos *Random Forest* y *BackPropagation*. El objetivo de este trabajo es comparar los resultados expuestos por cada método, así como sus respectivas prestaciones. Con este fin, intentaremos ajustar los hiperparámetros de cada uno de los algoritmos en busca de su mejor rendimiento.

# **PREPARACIÓN DE LOS DATOS**

+ Extraemos el path del DataFrame (DF).

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* Cargamos el DF

In [ ]:
df = pd.read_csv("/kaggle/input/mushroom-classification/mushrooms.csv")

print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1],"\n","\n" )
print(df.head())


Se muestran las dimensiones del conjunto, así como las primeras lineas del DF.

* Convertimos el conjunto de valores de cada atributo (strings) en valores numéricos.


In [ ]:
from sklearn import preprocessing

le=preprocessing.LabelEncoder()

for column in df.columns:
    df[column] = le.fit_transform(df[column])
        
print(df.head())



Se muestra las primeras lineas del DF después de dicha operación.

* Separamos las etiquetas de clase de las características de cada individuo dentro del DF:
        X: conjunto de individuos sin etiquetar
        Y: conjunto de etiquetas de clase

In [ ]:
X = df[['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']]
Y = df["class"]

print("conjunto de individuos sin etiquetar:","\n","\n",X,"\n","\n","\n","\n")
print("conjunto de etiquetas de clase:","\n","\n",Y,"\n")
 

+ Realizamos una partición del conjunto según las condiciones siguientes:
         El tamaño de los conjuntos de prueba y entrenamiento equivale respectivamente a uno y dos tercios del tamaño del DF original. 
         Separamos los conjuntos de prueba/entranamiento de manera aleatoria y fijamos dicha repartición.


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=1/3, random_state=0)

print("x_train:","\n","\n",x_train,"\n","\n","\n","\n")
print("x_test:","\n","\n",x_test,"\n","\n","\n","\n")
print("y_train:","\n","\n",y_train,"\n","\n","\n","\n")
print("y_test:","\n","\n",y_test,"\n")

La salida nos indica que el tamaño de los conjuntos de prueba y entrenamiento es respectivamente de 2708 y 5416 individuos.

 # **RANDOM FOREST**

+ Ajustamos el aprendizaje con los siguientes hiperparámetros:
        Número de árboles en el bosque: 10
        Árboles calculados en cada itéración: 3610 ( = int(2/3 * X.shape[0]) )
        
        
+ Entrenamos el clasificador con el conjunto previamente definido.

+ Sometemos el conjunto de prueba al clasificador.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf=RandomForestClassifier(n_estimators= 10,max_features = 'sqrt',max_samples = 2/3, random_state=0)

clf_rf.fit(x_train, y_train)

y_pred=clf_rf.predict(x_test)

print("y_pred:","\n","\n",y_test,"\n")

Se muestran las primeras lineas dela estimación de *Random Forest* sobre el conjunto de prueba.

+ Exhibimos la matriz de confusión y los principales estimadores.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusión:","\n ","\n ",confusion_matrix(y_test,y_pred),"\n ","\n ","\n ","\n ")

print("Rendimiento",":\n", classification_report(y_test,y_pred), "\n " )

Obtenemos estos resultados para un caso particular, en el que fijamos la repartición de conjuntos de prueba/entrenamiento y los árboles obtenidos por aprendizaje. Lo cierto es que cuando cuando corremos el código varias veces dejando variar estos dos parámetros, nos damos cuenta de que los tres indicadores (precisión, recall y f1) son iguales o muy próximos a 1 en la mayoría de los casos, hasta con bosques de un solo un árbol. A pesar de ello, decidimos priorizar la matriz de confusión a estos tres indicadores para configurar un bosque de 10 árboles. Y es que, al tratarse de un asunto sanitario y no un problema de optimización productiva, como sucede en otros casos, estimamos que el objetivo del algoritmo es el de minimizar el número de casos en los que un hongo venenoso se presenta como comestible. Con 10 árboles, la probabilidad de este caso es ínfima. Por último, dados que los resultados son satisfactorios con un bosque pequeño, el tiempo de ejecución del algoritmo es bastante corto. Rechazamos así la posibilidad de realizar podas, dejando de lado hyperparámetros como *max_depth*, *min_impurity_split* o *min_samples_split*.


# BACKPROPAGATION

La capa de entrada  22 neuronas (número de características de un hongo). La capa de salida posée una sola (output binario).

+ Ajustamos el aprendizaje con los siguientes hiperparámetros:
        Número de neuronas en la capa intermediaria: 100 
        Tipo de conexión utilizada: Total
        Función de activación: Rectificador
        Número máximo de ciclos de entrenamiento: 200
        Método de resolución: Método de cuasi-Newton
         
        
+ Entrenamos el clasificador con el conjunto previamente definido.

+ Sometemos el conjunto de prueba al clasificador.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_rn= MLPClassifier(hidden_layer_sizes=(100),activation="relu",solver="lbfgs", max_iter=200, random_state=0) 

clf_rn.fit(x_train, y_train)

y_pred_rn=clf_rn.predict(x_test)

print("y_pred_rn:","\n","\n",y_test,"\n")

Se muestran las primeras lineas de la estimación de *BackPropagation* sobre el conjunto de prueba.

+ Exhibimos la matriz de confusión y los principales estimadores junto con el error cuadrático medio.

In [ ]:
from sklearn.metrics import mean_squared_error

print("Matriz de confusión","\n","\n",confusion_matrix(y_test,y_pred_rn),"\n ","\n ","\n ","\n " )
print("Indicadores","\n","\n", classification_report(y_test,y_pred_rn),"\n ","\n ","\n ","\n " )
print("Error cuadrático medio:","\n","\n",mean_squared_error(y_test, y_pred_rn),"\n" )

En comparación a *Random Forest*, los hiperparámetros son más numerosos y su elección puede llegar a resultar poco fundamentada. Escogemos el método de activación de cuasi-Newton por ser sensiblemente más rápido, además de presentar una eficacidad más alta. Por esta última razón, elegimos también el rectificador como función de activación. En cuanto a la duración del entrenamiento, elegimos establecer un máximo de 200 ciclos para no limitar el aprendizaje. Variando el máximo de ciclos de entrenamiento, que desencadena un mensaje de alerta, conseguimos estimar el número medio de ciclos en torno a los 100. Al no introducir información sobre el tipo de conexión utilizado, suponemos que el perceptrón está totalmente conectado, suposición en coherencia con la aparente falta de hipótesis. Con respecto al número de neuronas de la capa intermediaria, su elección sea quizás la que más condicione el resultado. Con la comparación a *Random Forest* en mente, intentamos buscar un equilibrio entre exactitud de los resultados y velocidad de convergencia, fijando en 100 el número de neuronas. Aún así, *BackPropagation* resulta más lento y errático que *Random Forest*. A pesar de que la precisión, el recall y el f1 casi siempre son óptimos, el error cuadrático refleja un rendimiento inferior.

# CONCLUSIÓN

Como ya adelantábamos, el rendimiento sobre este set de *Random Forest* parece estar un paso por encima del de *BackPropagation*, tanto en exactitud como en velocidad. Eso sí, cabe destacar la habitual coincidencia entre los resultados presentados por ambos métodos.